# Profiling test

In [1]:
import sys
sys.path.append('../../../')

from pyrecdp.widgets import BaseWidget
view = BaseWidget()


Output(layout=Layout(border='1px solid black'))

In [2]:
def printhello():
    print("hello")
view.update(printhello)

In [3]:
view.clear()

# auto fe test

In [1]:
import sys
sys.path.append('../../../')

import pandas as pd
from utils import Timer

train_data = pd.read_csv("nyc_taxi_fare_cleaned.csv", nrows=1000)

from pyrecdp.autofe import FeatureWrangleGenerator
fwg = FeatureWrangleGenerator(dataset=train_data, label="fare_amount")

overide


In [2]:
print(fwg.get_transform_pipeline())

Stage 0: [<class 'pyrecdp.primitives.generators.astype.AsTypeFeatureGenerator'>]
Stage 1: [<class 'pyrecdp.primitives.generators.fillna.FillNaFeatureGenerator'>]
Stage 2: [<class 'pyrecdp.primitives.generators.identity.IdentityFeatureGenerator'>, <class 'pyrecdp.primitives.generators.datetime.DatetimeFeatureGenerator'>]
Stage 3: [<class 'pyrecdp.primitives.generators.drop_unique.DropUniqueFeatureGenerator'>]


In [3]:
fwg.get_feature_list()

fare_amount                  float64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
passenger_count                int64
pickup_datetime                int64
pickup_datetime.year           int64
pickup_datetime.month          int64
pickup_datetime.day            int64
pickup_datetime.dayofweek      int64
dtype: object

In [4]:
import sys
sys.path.append('../../../')

import pandas as pd
from utils import Timer

train_data = pd.read_csv("twitter_recsys_cleaned.csv", nrows=1000)

from pyrecdp.autofe import FeatureWrangleGenerator
fwg = FeatureWrangleGenerator(dataset=train_data, label="reply")

overide


In [5]:
print(fwg.get_transform_pipeline())

Stage 0: [<class 'pyrecdp.primitives.generators.astype.AsTypeFeatureGenerator'>]
Stage 1: [<class 'pyrecdp.primitives.generators.fillna.FillNaFeatureGenerator'>]
Stage 2: [<class 'pyrecdp.primitives.generators.identity.IdentityFeatureGenerator'>, <class 'pyrecdp.primitives.generators.category.CategoryFeatureGenerator'>, <class 'pyrecdp.primitives.generators.datetime.DatetimeFeatureGenerator'>]
Stage 3: [<class 'pyrecdp.primitives.generators.drop_unique.DropUniqueFeatureGenerator'>]


In [5]:
fwg.get_feature_list()

engaged_with_user_follower_count      int64
engaged_with_user_following_count     int64
engaged_with_user_is_verified          int8
engaging_user_follower_count          int64
enaging_user_following_count          int64
                                      ...  
__nlp__.28224                        uint16
__nlp__.4348                         uint16
__nlp__.56898                        uint16
__nlp__.56898 137                    uint16
__nlp__._total_                      uint16
Length: 212, dtype: object

# Backup

In [1]:
import pandas as pd
from utils import Timer
import os

def clean_df(df, label = 'reply'):
    df['tweet_timestamp'] = pd.to_datetime(df['tweet_timestamp'],unit='s')
    df['engaging_user_account_creation'] = pd.to_datetime(df['engaging_user_account_creation'],unit='s')
    df['engaged_with_user_account_creation'] = pd.to_datetime(df['engaged_with_user_account_creation'],unit='s')
    
    df['reply'] = df['reply_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['retweet'] = df['retweet_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['comment'] = df['retweet_with_comment_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['like'] = df['like_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df = df.drop(columns=['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp'])
    if 'tokens' in df.columns:
        df = df.drop(columns=['tokens'])
    return df

def apply_bert_decode(df):
    from transformers import BertTokenizer
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
    df["tweet"] = df["text_tokens"].progress_apply(lambda x: tokenizer.decode([int(n) for n in x.split('\t')]))
    df = df.drop(columns=['text_tokens'])
    return df

with Timer(f"read data"):
    schema = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains", "tweet_type",
        "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count", "engaged_with_user_following_count",
        "engaged_with_user_is_verified", "engaged_with_user_account_creation", "engaging_user_id", "engaging_user_follower_count",
        "enaging_user_following_count", "enaging_user_is_verified", "engaging_user_account_creation", "engagee_follows_engager",
        "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
    train_data = pd.read_csv("twitter_recsys.csv", sep='\x01', names = schema, header=None,)
with Timer(f"clean data"):
    train_data = clean_df(train_data)
with Timer("write_data"):
    train_data.to_csv("twitter_recsys_cleaned.csv", index = False)

read data took 82.04026912152767 sec
clean data took 7.953582388348877 sec
write_data took 219.43907960969955 sec
